In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
CHROME_DRIVER_PATH = "" #PATH TO YOUR CHROME DRIVER CHANGE THIS LINE
URL = "https://store.steampowered.com/specials?snr=1_4_4__125#tab=TopSellers"

In [11]:
options = Options()
options.add_argument("start-maximized")
#options.add_argument("--headless")

In [12]:
def sales_from_steam():
    driver = webdriver.Chrome(service=Service(CHROME_DRIVER_PATH), options=options)
    print(f"Navigating to: {URL}")
    driver.get(URL)

    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'SaleSection_13268'))
        )

        # Scroll to bottom to load all items
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Find all game containers
        game_containers = driver.find_elements(By.CSS_SELECTOR, 'div.v9uRg57bwOaPsvAnkXESO')

        games = []

        for game in game_containers:
            try:
                title = game.find_element(By.CSS_SELECTOR, 'div.StoreSaleWidgetTitle').text
                discount = game.find_element(By.CSS_SELECTOR, 'div.cnkoFkzVCby40gJ0jGGS4').text
                
                # Try to get prices (some games may not have both old and new prices)
                try:
                    old_price = game.find_element(By.CSS_SELECTOR, 'div._3fFFsvII7Y2KXNLDk_krOW').text
                    new_price = game.find_element(By.CSS_SELECTOR, 'div._3j4dI1yA7cRfCvK8h406OB').text
                except:
                    old_price = new_price = "N/A"

                games.append({
                    'title': title,
                    'discount': discount,
                    'old_price': old_price,
                    'new_price': new_price
                })

            except Exception as e:
                print(f"Error parsing a game: {e}")
                continue
        return games

    except Exception as e:
        print(f"Error: {e}")
    finally:
        driver.quit()


In [13]:
def search_game_by_price(game_name, target_price):
    driver = webdriver.Chrome(service=Service(CHROME_DRIVER_PATH), options=options)
    
    try:
        # Navigate to Steam store
        driver.get('https://store.steampowered.com/')

        # Wait for the search box to be present
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'store_nav_search_term'))
        )

        # Locate the search box and type the game name
        search_box = driver.find_element(By.ID, 'store_nav_search_term')
        search_box.clear()
        search_box.send_keys(game_name)
        search_box.send_keys(Keys.RETURN)  # Press Enter to submit the search

        # Wait until the results are loaded (check for visibility of the game titles)
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.responsive_search_name_combined'))
        )

        # Find the first result (game title and price)
        first_result = driver.find_element(By.CSS_SELECTOR, 'div.responsive_search_name_combined')

        # Extract the game title
        title = first_result.find_element(By.CSS_SELECTOR, 'span.title').text.strip()

        # Extract price information
        price_block = first_result.find_element(By.CSS_SELECTOR, 'div.search_price_discount_combined')
        
        try:
            # Check if the game has a discount
            discount_block = price_block.find_element(By.CSS_SELECTOR, 'div.discount_block')

            if 'no_discount' in discount_block.get_attribute('class'):
                # No discount, get the regular price
                final_price_element = price_block.find_element(By.CSS_SELECTOR, 'div.discount_final_price')
                final_price = final_price_element.text.strip()

                # Convert the final price to a float (remove currency symbols and replace commas)
                final_price = float(final_price.replace('€', '').replace(',', '.').strip())
            else:
                # Discounted game, get the final discounted price
                final_price_element = discount_block.find_element(By.CSS_SELECTOR, 'div.discount_final_price')
                final_price = final_price_element.text.strip()

                # Convert the final price to a float (remove currency symbols and replace commas)
                final_price = float(final_price.replace('€', '').replace(',', '.').strip())

        except Exception as e:
            print(f"Error extracting price for {title}: {e}")
            return None

        # Compare the price with the target price
        if final_price <= target_price:
            print(f"Game: {title}")
            print(f"Price: {final_price}€")
            return title, final_price

        print(f"Game: {title} is not under the target price.")
        return None

    except Exception as e:
        print(f"Error during search: {e}")
        return None
    
    finally:
        driver.quit()

In [14]:
sales_from_steam()

Navigating to: https://store.steampowered.com/specials?snr=1_4_4__125#tab=TopSellers


[{'title': "Baldur's Gate 3",
  'discount': '-20%',
  'old_price': '59,99€',
  'new_price': '47,99€'},
 {'title': 'Devil May Cry 5',
  'discount': '-75%',
  'old_price': '29,99€',
  'new_price': '7,49€'},
 {'title': 'Red Dead Redemption 2',
  'discount': '-75%',
  'old_price': '59,99€',
  'new_price': '14,99€'},
 {'title': 'Devil May Cry HD Collection',
  'discount': '-67%',
  'old_price': '29,99€',
  'new_price': '9,89€'},
 {'title': 'Dead by Daylight',
  'discount': '-60%',
  'old_price': '19,99€',
  'new_price': '7,99€'},
 {'title': 'Sons Of The Forest',
  'discount': '-60%',
  'old_price': '28,99€',
  'new_price': '11,59€'},
 {'title': 'Commandos: Origins',
  'discount': '-10%',
  'old_price': '49,99€',
  'new_price': 'Prepurchase'},
 {'title': 'Devil May Cry 4 Special Edition',
  'discount': '-70%',
  'old_price': '24,99€',
  'new_price': '7,49€'},
 {'title': 'Euro Truck Simulator 2',
  'discount': '-75%',
  'old_price': '19,99€',
  'new_price': '4,99€'},
 {'title': 'Call of Duty®

In [15]:
search_game_by_price('Devil May Cry 5', 10.00)

Game: Devil May Cry 5
Price: 7.49€


('Devil May Cry 5', 7.49)

In [16]:
search_game_by_price('hogwarts legacy', 40)

Game: Hogwarts Legacy is not under the target price.
